In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/LLM/Gen_AI')

In [4]:
# """!pip install transformers
# !pip install datasets
# !pip install tokenizer
# !pip install seqeval
# """
# #either you can use above statement or use single command
# # Install
!pip install transformers datasets tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 12.6 MB/s eta 0:00:00


In [5]:
!pip install datasets

In [3]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification

In [4]:
conll2003 = datasets.load_dataset("conll2003")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [7]:
conll2003["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [8]:
conll2003["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [8]:
conll2003["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [ ]:
conll2003["train"].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [9]:
conll2003["train"].description

'The shared task of CoNLL-2003 concerns language-independent named entity recognition. We will concentrate on\nfour types of named entities: persons, locations, organizations and names of miscellaneous entities that do\nnot belong to the previous three groups.\n\nThe CoNLL-2003 shared task data files contain four columns separated by a single space. Each word has been put on\na separate line and there is an empty line after each sentence. The first item on each line is a word, the second\na part-of-speech (POS) tag, the third a syntactic chunk tag and the fourth the named entity tag. The chunk tags\nand the named entity tags have the format I-TYPE which means that the word is inside a phrase of type TYPE. Only\nif two phrases of the same type immediately follow each other, the first word of the second phrase will have tag\nB-TYPE to show that it starts a new phrase. A word with tag O is not part of a phrase. Note the dataset uses IOB2\ntagging scheme, whereas the original dataset uses 

In [5]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
conll2003['train'][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [ ]:
conll2003["train"].features['ner_tags']

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [11]:
example_text = conll2003['train'][10]
example_text

{'id': '10',
 'tokens': ['Spanish',
  'Farm',
  'Minister',
  'Loyola',
  'de',
  'Palacio',
  'had',
  'earlier',
  'accused',
  'Fischler',
  'at',
  'an',
  'EU',
  'farm',
  'ministers',
  "'",
  'meeting',
  'of',
  'causing',
  'unjustified',
  'alarm',
  'through',
  '"',
  'dangerous',
  'generalisation',
  '.',
  '"'],
 'pos_tags': [22,
  22,
  22,
  22,
  22,
  22,
  38,
  31,
  40,
  22,
  15,
  12,
  16,
  21,
  24,
  27,
  21,
  15,
  39,
  16,
  21,
  15,
  0,
  16,
  21,
  7,
  0],
 'chunk_tags': [11,
  12,
  12,
  12,
  12,
  12,
  21,
  22,
  22,
  11,
  13,
  11,
  12,
  12,
  12,
  11,
  12,
  13,
  21,
  1,
  11,
  13,
  0,
  11,
  12,
  0,
  0],
 'ner_tags': [7,
  0,
  0,
  1,
  2,
  2,
  0,
  0,
  0,
  1,
  0,
  0,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [13]:
example_text["tokens"]

['Spanish',
 'Farm',
 'Minister',
 'Loyola',
 'de',
 'Palacio',
 'had',
 'earlier',
 'accused',
 'Fischler',
 'at',
 'an',
 'EU',
 'farm',
 'ministers',
 "'",
 'meeting',
 'of',
 'causing',
 'unjustified',
 'alarm',
 'through',
 '"',
 'dangerous',
 'generalisation',
 '.',
 '"']

In [14]:
tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

Here's a breakdown of token_type_ids:

Purpose: This list helps some NLP models, particularly BERT, understand if there are multiple pieces of text (sentences) being provided as input.<br>
Values: Each element in the list corresponds to a word (token) in the input_ids list. The value can be either 0 or 1.<br>
Interpretation in your case: Since all the values in token_type_ids are 0, the model understands that all the tokens (words) belong to the same sentence. There's no distinction between separate pieces of text.<br>

Points to note:<br>
Not all NLP models require token_type_ids. Some models can infer sentence boundaries from other information like punctuation or special tokens.<br>
The use of token_type_ids can become relevant when dealing with tasks involving two or more sentences, like question answering or sentiment analysis of a conversation between two people. In such cases, you would see a combination of 0s and 1s in the token_type_ids list to differentiate between the sentences.

In [15]:
tokenized_input

{'input_ids': [101, 3009, 3888, 2704, 21580, 2139, 14412, 21361, 2018, 3041, 5496, 27424, 2818, 3917, 2012, 2019, 7327, 3888, 7767, 1005, 3116, 1997, 4786, 4895, 29427, 7810, 8598, 2083, 1000, 4795, 2236, 6648, 1012, 1000, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [16]:
tokenized_input["input_ids"]

[101,
 3009,
 3888,
 2704,
 21580,
 2139,
 14412,
 21361,
 2018,
 3041,
 5496,
 27424,
 2818,
 3917,
 2012,
 2019,
 7327,
 3888,
 7767,
 1005,
 3116,
 1997,
 4786,
 4895,
 29427,
 7810,
 8598,
 2083,
 1000,
 4795,
 2236,
 6648,
 1012,
 1000,
 102]

In [20]:
tokenized_input

{'input_ids': [101, 3009, 3888, 2704, 21580, 2139, 14412, 21361, 2018, 3041, 5496, 27424, 2818, 3917, 2012, 2019, 7327, 3888, 7767, 1005, 3116, 1997, 4786, 4895, 29427, 7810, 8598, 2083, 1000, 4795, 2236, 6648, 1012, 1000, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

tokenizer.convert_ids_to_tokens function:

This function is part of libraries like Transformers used for Natural Language Processing (NLP) tasks.<br>
It takes a list of integer IDs (typically from the input_ids field after tokenization) as input.<br>
Its purpose is to convert these IDs back into the corresponding tokens (words) in the original vocabulary used by the tokenizer.

In [17]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [18]:
tokens

['[CLS]',
 'spanish',
 'farm',
 'minister',
 'loyola',
 'de',
 'pal',
 '##acio',
 'had',
 'earlier',
 'accused',
 'fis',
 '##ch',
 '##ler',
 'at',
 'an',
 'eu',
 'farm',
 'ministers',
 "'",
 'meeting',
 'of',
 'causing',
 'un',
 '##just',
 '##ified',
 'alarm',
 'through',
 '"',
 'dangerous',
 'general',
 '##isation',
 '.',
 '"',
 '[SEP]']

In [19]:
word_ids = tokenized_input.word_ids()

print(word_ids)

[None, 0, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 9, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 20, 21, 22, 23, 24, 24, 25, 26, None]


In [ ]:
example_text["ner_tags"]

[7,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
for i, label in enumerate(example_text["ner_tags"]):
  print(i,label)

0 3
1 0
2 7
3 0
4 0
5 0
6 7
7 0
8 0


label_all_tokens=True (default):

In this case, the code assigns the NER label to all tokens, including those that might be subwords created during tokenization.<br>
This means that even if a single word is split into multiple subwords, each subword will inherit the NER label of the entire word.
label_all_tokens=False:

When set to False, the code only assigns the NER label to the first subword of a word that gets split during tokenization.<br>
Subsequent subwords representing the same word will be assigned a label of -100, essentially ignoring them in the loss function during training.

Why the difference?

Some tokenizers, like Byte Pair Encoding (BPE), might break down words into smaller subword units.<br>
Assigning labels to every subword can be redundant, especially if the subword doesn't carry much meaning on its own.<br>
By assigning the label only to the first subword and ignoring the rest (with -100), the model focuses on the most relevant parts of the word for NER tasks.<br>
Example:

Consider the word "running" which might be split into subwords "run" and "ning" by the tokenizer.

With label_all_tokens=True, both "run" and "ning" would get the same NER label assigned (assuming "running" has a label).<br>
With label_all_tokens=False, only "run" would get the NER label, and "ning" would be assigned -100, effectively ignoring it during training.<br>
Choosing the right setting:

If your task focuses on recognizing named entities at the word level, label_all_tokens=False might be more efficient.<br>
If you need to capture information within subwords (less common), label_all_tokens=True might be necessary.<br>
The best setting depends on your specific NER task and the tokenizer you're using

The batch_index argument in the word_ids method is used because the tokenize_and_align_labels function is likely designed to handle multiple examples (sentences) at once, forming a batch.

Here's a breakdown of why batch_index is important:

Batch processing: NLP libraries like Transformers often allow processing multiple text examples in a single call for efficiency. This creates a batch of data.<br>
tokenized_inputs: This variable likely holds the results of tokenization for the entire batch of examples. It might be a dictionary containing information like token IDs, attention masks, etc., for each example.<br>
word_ids: This method retrieves the word IDs specifically for a particular example within the batch. The batch_index argument tells the function which example's word IDs you want to access.<br>
Example:

Imagine you have two sentences (examples) and you tokenize them together:

Sentence 1: "The quick brown fox jumps over the lazy dog"<br>
Sentence 2: "This is a test sentence"<br>
After tokenization, tokenized_inputs might be a dictionary containing token IDs and other information for both sentences.

When you use word_ids(batch_index=0), you're accessing the word IDs for the first sentence (batch index 0) within tokenized_inputs. Similarly, word_ids(batch_index=1) would retrieve the word IDs for the second sentence.

In essence, batch_index allows you to extract information (like word IDs) for a specific example within a batch of processed text data.

In [6]:
def tokenize_and_align_labels(examples, label_all_tokens=False):

    #tokeinze ids
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []

    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)#
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.

        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)

            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                #print(label)
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
conll2003["train"][10:11]

{'id': ['10'],
 'tokens': [['Spanish',
   'Farm',
   'Minister',
   'Loyola',
   'de',
   'Palacio',
   'had',
   'earlier',
   'accused',
   'Fischler',
   'at',
   'an',
   'EU',
   'farm',
   'ministers',
   "'",
   'meeting',
   'of',
   'causing',
   'unjustified',
   'alarm',
   'through',
   '"',
   'dangerous',
   'generalisation',
   '.',
   '"']],
 'pos_tags': [[22,
   22,
   22,
   22,
   22,
   22,
   38,
   31,
   40,
   22,
   15,
   12,
   16,
   21,
   24,
   27,
   21,
   15,
   39,
   16,
   21,
   15,
   0,
   16,
   21,
   7,
   0]],
 'chunk_tags': [[11,
   12,
   12,
   12,
   12,
   12,
   21,
   22,
   22,
   11,
   13,
   11,
   12,
   12,
   12,
   11,
   12,
   13,
   21,
   1,
   11,
   13,
   0,
   11,
   12,
   0,
   0]],
 'ner_tags': [[7,
   0,
   0,
   1,
   2,
   2,
   0,
   0,
   0,
   1,
   0,
   0,
   3,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0]]}

In [ ]:
q=tokenize_and_align_labels(conll2003["train"][10:11])

In [30]:
q

{'input_ids': [[101, 3009, 3888, 2704, 21580, 2139, 14412, 21361, 2018, 3041, 5496, 27424, 2818, 3917, 2012, 2019, 7327, 3888, 7767, 1005, 3116, 1997, 4786, 4895, 29427, 7810, 8598, 2083, 1000, 4795, 2236, 6648, 1012, 1000, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 7, 0, 0, 1, 2, 2, 2, 0, 0, 0, 1, 1, 1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [ ]:
conll2003["train"][4:5]['ner_tags']

[[5,
  0,
  0,
  0,
  0,
  3,
  4,
  0,
  0,
  0,
  1,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  5,
  0,
  0,
  0,
  0,
  0,
  0,
  0]]

In [31]:
q=tokenize_and_align_labels(conll2003["train"][4:5])

In [32]:
q

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}

In [33]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
germany_________________________________ 5
'_______________________________________ 0
s_______________________________________ 0
representative__________________________ 0
to______________________________________ 0
the_____________________________________ 0
european________________________________ 3
union___________________________________ 4
'_______________________________________ 0
s_______________________________________ 0
veterinary______________________________ 0
committee_______________________________ 0
werner__________________________________ 1
z_______________________________________ 2
##wing__________________________________ 2
##mann__________________________________ 2
said____________________________________ 0
on______________________________________ 0
wednesday_______________________________ 0
consumers_______________________________ 0
should__________________________________ 0
buy_____________________________________ 0
sheep___

In [7]:
## Applying on entire data
tokenized_datasets=conll2003.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

In [8]:
tokenized_datasets["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 14041
})

In [8]:
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",num_labels=9)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments, Trainer
data_collator=DataCollatorForTokenClassification(tokenizer)

In [19]:
args = TrainingArguments(
"test-ner",
evaluation_strategy = "epoch",
learning_rate=2e-5,
per_device_train_batch_size=16,
per_device_eval_batch_size=16,
num_train_epochs=10,
weight_decay=0.01
)

In [11]:
#!pip install transformers[torch]

1. Batching for Efficient Training:

Deep learning models like Transformers typically train on batches of data rather than individual examples. This improves training speed and memory usage on your hardware.<br>
Data collators act as the bridge between your raw data and the training process. They take care of assembling individual data points (e.g., tokenized sentences, labels) into these training batches.
2. Data Processing and Alignment:

Data collators can perform various processing steps on your data before batching. This might include:<br>
Padding: Ensuring all elements in a batch have the same length. Models that process sequences require fixed input sizes. Padding achieves this by adding special tokens (e.g., "[PAD]") to shorter sequences.<br>
Truncation: Limiting the length of overly long sequences to fit training constraints.<br>
Masking: Techniques like random masking (used in language modeling) can be applied by data collators.<br>
3. Handling Different Data Formats:

NLP tasks often involve different data formats. Data collators can handle this variety. They allow you to specify how various elements (tokens, labels, etc.) in your data should be combined into a training batch.

In [21]:
# Trainer(
#    model,
#    args,
#    train_dataset=tokenized_datasets["train"],
#    eval_dataset=tokenized_datasets["validation"],
#    data_collator=data_collator,
#    tokenizer=tokenizer,
#    compute_metrics=compute_metrics
# )

In [12]:
metric=datasets.load_metric("seqeval")

<ipython-input-12-453905903fc4>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric=datasets.load_metric("seqeval")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for seqeval contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/seqeval/seqeval.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [13]:
conll2003

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [23]:
example=conll2003['train'][0]

In [26]:
example

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [26]:
label_list = conll2003["train"].features["ner_tags"].feature.names

label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [27]:
labels = [label_list[i] for i in example["ner_tags"]]
labels

NameError: name 'example' is not defined

In [25]:
metric.compute(predictions=[labels],references=[labels])

{'MISC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [36]:
value=0

In [41]:

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)

    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
          "precision": results["overall_precision"],
          "recall": results["overall_recall"],
          "f1": results["overall_f1"],
          "accuracy": results["overall_accuracy"],
  }

In [42]:
data_collator=DataCollatorForTokenClassification(tokenizer)

In [43]:
trainer=Trainer(
   model,
   args,
   train_dataset=tokenized_datasets["test"],
   eval_dataset=tokenized_datasets["validation"],
   data_collator=data_collator,
   tokenizer=tokenizer,
   compute_metrics=compute_metrics
)

In [44]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.134518,0.901658,0.915012,0.908286,0.980900
2,No log,0.130071,0.892074,0.911141,0.901507,0.980900
3,0.003800,0.131711,0.906953,0.908785,0.907868,0.981309
4,0.003800,0.136192,0.903811,0.914002,0.908878,0.981329
5,0.004300,0.131922,0.897151,0.911646,0.904341,0.981114
6,0.004300,0.135502,0.910951,0.914170,0.912558,0.981757
7,0.002100,0.134559,0.899309,0.919892,0.909484,0.981348
8,0.002100,0.136444,0.898908,0.914339,0.906558,0.981426
9,0.002100,0.142117,0.905906,0.913834,0.909853,0.981699
10,0.001000,0.140406,0.905563,0.915012,0.910263,0.981893


TrainOutput(global_step=2160, training_loss=0.002696328810243695, metrics={'train_runtime': 483.7322, 'train_samples_per_second': 71.382, 'train_steps_per_second': 4.465, 'total_flos': 848476607721174.0, 'train_loss': 0.002696328810243695, 'epoch': 10.0})

In [45]:
model.save_pretrained("ner_model")

In [46]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [47]:
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [48]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}

In [49]:
id2label

{'0': 'O',
 '1': 'B-PER',
 '2': 'I-PER',
 '3': 'B-ORG',
 '4': 'I-ORG',
 '5': 'B-LOC',
 '6': 'I-LOC',
 '7': 'B-MISC',
 '8': 'I-MISC'}

In [50]:
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [51]:
label2id

{'O': '0',
 'B-PER': '1',
 'I-PER': '2',
 'B-ORG': '3',
 'I-ORG': '4',
 'B-LOC': '5',
 'I-LOC': '6',
 'B-MISC': '7',
 'I-MISC': '8'}

In [52]:
import json

In [54]:
config=json.load(open("/content/drive/MyDrive/LLM/Gen_AI/ner_model/config.json"))

In [55]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'LABEL_0',
  '1': 'LABEL_1',
  '2': 'LABEL_2',
  '3': 'LABEL_3',
  '4': 'LABEL_4',
  '5': 'LABEL_5',
  '6': 'LABEL_6',
  '7': 'LABEL_7',
  '8': 'LABEL_8'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'LABEL_0': 0,
  'LABEL_1': 1,
  'LABEL_2': 2,
  'LABEL_3': 3,
  'LABEL_4': 4,
  'LABEL_5': 5,
  'LABEL_6': 6,
  'LABEL_7': 7,
  'LABEL_8': 8},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.40.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [56]:
config["id2label"] = id2label

In [57]:
config["label2id"] = label2id

In [58]:
config

{'_name_or_path': 'bert-base-uncased',
 'architectures': ['BertForTokenClassification'],
 'attention_probs_dropout_prob': 0.1,
 'classifier_dropout': None,
 'gradient_checkpointing': False,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'id2label': {'0': 'O',
  '1': 'B-PER',
  '2': 'I-PER',
  '3': 'B-ORG',
  '4': 'I-ORG',
  '5': 'B-LOC',
  '6': 'I-LOC',
  '7': 'B-MISC',
  '8': 'I-MISC'},
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'label2id': {'O': '0',
  'B-PER': '1',
  'I-PER': '2',
  'B-ORG': '3',
  'I-ORG': '4',
  'B-LOC': '5',
  'I-LOC': '6',
  'B-MISC': '7',
  'I-MISC': '8'},
 'layer_norm_eps': 1e-12,
 'max_position_embeddings': 512,
 'model_type': 'bert',
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'pad_token_id': 0,
 'position_embedding_type': 'absolute',
 'torch_dtype': 'float32',
 'transformers_version': '4.40.2',
 'type_vocab_size': 2,
 'use_cache': True,
 'vocab_size': 30522}

In [59]:
json.dump(config,open("/content/drive/MyDrive/LLM/Gen_AI/ner_model/config.json","w"))

In [65]:
ls

huggingface_transformer.ipynb  ner_model/  test-ner/  tokenizer/  train-ner/


In [60]:
model_fine_tuned=AutoModelForTokenClassification.from_pretrained("ner_model")

In [61]:
model_fine_tuned

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

# transformer pipeline

In [62]:
from transformers import pipeline

In [63]:
nlp_pipeline=pipeline("ner",model=model_fine_tuned,tokenizer=tokenizer)

In [64]:
nlp_pipeline

In [66]:
example="Riad is not able to figure out what to do in his life. He is hopeless. Only One can help him"

In [67]:
nlp_pipeline(example)

[{'entity': 'B-PER',
  'score': 0.9997371,
  'index': 1,
  'word': 'ri',
  'start': 0,
  'end': 2},
 {'entity': 'B-PER',
  'score': 0.99857414,
  'index': 2,
  'word': '##ad',
  'start': 2,
  'end': 4}]

In [76]:
example="Bangladesh is a peaceful country"

In [77]:
nlp_pipeline(example)

[{'entity': 'B-LOC',
  'score': 0.99979097,
  'index': 1,
  'word': 'bangladesh',
  'start': 0,
  'end': 10}]

In [74]:
example="apple launch mobile while eating apple which taste like orange"


In [75]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.98410773,
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5},
 {'entity': 'B-MISC',
  'score': 0.98444194,
  'index': 6,
  'word': 'apple',
  'start': 33,
  'end': 38}]

In [78]:
example="apple founder loves eating apple"

In [79]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.9994856,
  'index': 1,
  'word': 'apple',
  'start': 0,
  'end': 5}]

In [80]:
example="Microsoft Windows created their software by idea that came from the window of the house"


In [81]:
nlp_pipeline(example)

[{'entity': 'B-ORG',
  'score': 0.99952674,
  'index': 1,
  'word': 'microsoft',
  'start': 0,
  'end': 9},
 {'entity': 'I-ORG',
  'score': 0.99924207,
  'index': 2,
  'word': 'windows',
  'start': 10,
  'end': 17}]